# Multiple-Instance Learning with Yelp Restaurant Dataset

This notebook aims to train a classifier for the business IDs using the aggregated features extracted from the final dense layer of a VGG16 network pre-trained on ImageNet data. You can download the weights [here](https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view?usp=sharing), courtesy of [baraldilorenzo](https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3).


## 1) Extraction of Image Features using pre-trained VGG16 model in Keras

To begin, we need to download the model weights and load them.

In [1]:

import numpy as np
import pandas as pd
import h5py
from time import time
from PIL import Image

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D


# build VGG-16 model
def VGG_16(weights_path=None):

    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    # remove final softmax layer
    fc7 = Sequential()
    depth = len(model.layers)
    for i in range(depth-1):
       fc7.add(model.layers[i])
    # compile to get predict function. arbitrary optimizer and loss ok
    fc7.compile(optimizer='sgd',loss='mse') 

    return fc7

# build model
model = VGG_16('models/vgg16_weights.h5')

Using Theano backend.
Using gpu device 0: Tesla K40c (CNMeM is disabled, CuDNN not available)
/opt/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


Next, we read in the image data, and store the features from the final dense layer (4096-dim). 
Takes over 12 hours using a GPU for the test set.

In [ ]:
_,ch,h,w = model.input_shape
_,nf = model.output_shape

# store training features
def store_feat(is_train, batch_size=128):
    t_read = time()
    if is_train:
        h5_path, photo_path = 'data/train_feat.h5','data/train_photos'
        photo_biz = pd.read_csv('data/train_photo_to_biz_ids.csv')
    else:
        h5_path, photo_path = 'data/test_feat.h5','data/test_photos'
        photo_biz = pd.read_csv('data/test_photo_to_biz.csv')
    mean_pixels = np.array([123.68,116.779,103.939]).reshape((3,1,1))
    feat = h5py.File(h5_path,'w')
    X = feat.create_dataset(
        name='X',
        shape=(len(photo_biz),nf),
        dtype=np.float32)
    for b in range(len(photo_biz)/batch_size+1):
        i_lo, i_hi = b*batch_size, min((b+1)*batch_size,len(photo_biz))
        batch_size_i = i_hi - i_lo
        raw_i = np.zeros((batch_size_i,ch,h,w))
        for i in range(i_lo,i_hi):
            f_ = '%s/%s.jpg' % (photo_path,photo_biz['photo_id'][i])
            im_j = Image.open(f_).resize((w,h))
            ima_j = np.asarray(im_j).transpose((2,0,1))
            raw_i[i-i_lo,:,:,:] = ima_j - mean_pixels
        X[i_lo:i_hi,:] = model.predict(raw_i,batch_size_i)
        if i_hi % 1024 == 0:
            print('reading %ith photo took %ds' % (i_hi,time()-t_read))
    feat.close()

store_feat(is_train=True)
store_feat(is_train=False)

## 2) Aggregate Features by Business ID

We perform a simple aggregation of our data over the business ID's by taking the mean, std, min, max of each of the 4096 features, and add a count of the number of pictures associated with the business. That should give us a (2000 by 4096x4+1) matrix. 

Let's begin by reading in our data.

In [2]:
# read in csv data
train_biz_lab = pd.read_csv('data/train.csv')
test_biz_lab = pd.read_csv('data/sample_submission.csv')
train_photo_biz = pd.read_csv('data/train_photo_to_biz_ids.csv')
test_photo_biz = pd.read_csv('data/test_photo_to_biz.csv')

We define a function for calculating the various stats of the 4096 features for each business ID. That should give us a (2000 by 4096x4+1) and a (10000 by 4096x4+1) matrix for the train and test sets respectively.

Let's perform the aggregation over the business_ids, and save the aggregated feature information in another hdf5 file. Takes about 7 minutes on my red hat server.

In [ ]:
# function for aggregating feature data
def get_agg(is_train):
    t_start = time()
    if is_train:
        print('Aggregating for training data')
        feat_path, agg_path = 'data/train_feat.h5', 'data/train_agg_feat.h5'
        photo_biz, biz_lab = train_photo_biz, train_biz_lab
    else:
        print('Aggregating for testing data')
        feat_path, agg_path = 'data/test_feat.h5', 'data/test_agg_feat.h5'
        photo_biz, biz_lab = test_photo_biz, test_biz_lab
    nrow, ncol = len(biz_lab), 4*4096+1
    feat = h5py.File(feat_path,'r')
    agg = h5py.File(agg_path,'w')
    Xa = agg.create_dataset(
        name='X',
        shape=(nrow,ncol),
        dtype=np.float32)
    for i in biz_lab.index:
        biz_i = biz_lab['business_id'][i]
        photo_ix = photo_biz.index[photo_biz['business_id']==biz_i]
        biz_mean = feat['X'][photo_ix].mean(axis=0)
        biz_std = feat['X'][photo_ix].std(axis=0)
        biz_max = feat['X'][photo_ix].max(axis=0)
        biz_min = feat['X'][photo_ix].min(axis=0)
        Xa[i,:] = np.hstack([biz_mean,biz_std,biz_max,biz_min,len(photo_ix)])
        if i % 200 == 0:
            print('Done with %dth business' % i)
    feat.close()
    agg.close()
    print('Took %ds' % (time()-t_start))

get_agg(is_train=True)
get_agg(is_train=False)

## 3) Train a SVM on the aggregated features

Having extracted and aggregated the features, we now train a multi-label classifier with some nice functions from sklearn. For easy re-use, we wrap our cross-validation with svm in a function to facilitate easy training with different parameter sets, especially if you're planning to parallelize the parameter tuning.

Some things to take note:
 - We have a case where n_examples (2k) << n_features (16k). As such we might not need to 'project' into high dimensional feature space using a rbf kernel. Moreover our features also captures some second-order statistics. A useful guide [here](www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf) has more tips on training a SVM classifier.
 - Smaller C corresponds to more regularization
 - You can choose to scale or not, but the optimal parameters may differ depending on that choice. linear kernels aren't that sensitive to scaling, while rbf kernels typically require it to work well with the default parameters (gamma).
 - Each model takes about 6mins+ to train on my red hat server using 1 thread. A 5-fold cv for a set of parameters would take about over half an hour to run. Feel free to parallelize it by spawning multiple processes or distributing it to a cluster.

In [ ]:
# import sklearn functions
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import f1_score

# read in training data from hdf5
feat = h5py.File('data/train_agg_feat.h5','r')
X = feat['X'][...]
feat.close()

# read labels for training data
def to_bool(x):
    return(pd.Series([1L if str(i) in str(x).split(' ') else 0L for i in range(9)]))
y = np.asarray(train_biz_lab['labels'].apply(to_bool))

# function for cross-validating svm
def svm_cv(X,y,kernel='linear',C=.1,scale=False,nfolds=5,save_path=None):
    scaler = StandardScaler()
    clf = OneVsRestClassifier(svm.SVC(kernel=kernel,C=C, random_state=290615))
    cv = KFold(n=len(X),n_folds=nfolds,shuffle=True,random_state=290615)
    F1cv = np.zeros(nfolds)
    for i, (a,b) in enumerate(cv):
        t_i = time()
        if scale:
            X_train = scaler.fit_transform(X[a])
            X_test = scaler.transform(X[b])
        else:
            X_train = X[a]
            X_test = X[b]
        y_train = y[a]
        y_test = y[b]
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        F1cv[i] = f1_score(y_test, y_pred, average='micro')
        print('F1 per class:' + str(f1_score(y_test, y_pred, average=None)))
        print('Fold %d F1 %.3f took %ds' % (i, F1cv[i], time()-t_i))
    print('Average F1 was %.3f' % (F1cv.mean()))
    # append cv results to csv file and save it
    if save_path:
        cv_res = pd.DataFrame({'kernel':kernel,'C':C,'nfolds':nfolds,'scale':scale,'F1cv':[F1cv],'F1':F1cv.mean()})
        cv_history = pd.read_csv('data/cv_history.csv')
        cv_history = cv_history.append(cv_res)
        cv_history.to_csv('data/cv_history.csv',index=False)


We can now train various combinations by calling the function in an interactive fashion. Here's the best one that worked for me :)

In [ ]:
svm_cv(X,y,kernel='rbf',C=3,scale=True,save_path='data/cv_history.csv') # avg F1 = 0.769

Now that we've cross-validated our model parameters, let's predict on the test set (submission data)!

In [ ]:
# fit on full training data
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)
clf = OneVsRestClassifier(svm.SVC(kernel='rbf',C=3., random_state=290615))
clf.fit(X_sc,y)

# read in aggregated features from hdf5 # ~ 39 mins
feat = h5py.File('data/test_agg_feat.h5','r')
X_test = scaler.transform(feat['X'][...])
feat.close()
y_pred = clf.predict(X_test)

# convert boolean matrix to integer labels
y_int = np.where(y_pred)
df = pd.DataFrame({'biz':y_int[0],'int':y_int[1]})
test_biz_lab['labels'] = df.groupby('biz').apply(lambda x: ' '.join([str(i) for i in x['int']]))
test_biz_lab.to_csv('data/sub5_vgg_svm.csv',index=False)